# Here I will deal mostly with extracting data from PDF files

### In accompanying scripts, I convert SAS and mutli-sheet Excel files into csv files for easy use in pandas using sas7bdat and pandas itself

In [1]:
# Necessary imports
import csv
import matplotlib.pyplot as plt
import numpy as np
import os
import openpyxl
import pandas as pd
import pdfplumber
import PyPDF2
import re
import tabula
#import sas7bdat
from openpyxl import load_workbook
from pathlib import Path
from copy import deepcopy
from tabula import read_pdf

##### Possibly handy functions

In [2]:
def normalize_whitespace(s, replace):
    if isinstance(s, str):
        return re.sub(r'\s+', replace, s)
    else:
        return s

def convert_multisheet(filepath, csv_path=''):
    prefix = Path(filepath).stem
    wb = load_workbook(filepath)
    sheets = wb.get_sheet_names()
    for sheet in sheets[1:]:
        df = wb[sheet]
        with open(f'{csv_path}{prefix}{sheet}.csv', 'w', newline='') as csvfile:
            c = csv.writer(csvfile)
            data =[]
            for row in df.rows:
                data.append([cell.value for cell in row])
            c.writerows(data)
        print(f'{prefix}{sheet}.csv file written')

##### Probably useful definitions

In [3]:
boroughs_ = {
    'Manhattan': ['1','MN', 100, 12],
    'Brooklyn': ['3', 'BK', 300, 18],
    'Bronx': ['2', 'BX', 200, 12],
    'Queens': ['4', 'QN', 400, 14],
    'Staten Island': ['5', 'SI', 500, 3]
}

borough_labels = {'Manhattan': [],
                  'Brooklyn': [],
                  'Bronx': [],
                  'Queens': [],
                  'Staten Island': []}

for borough, abbr in boroughs_.items():
    for n in range(1, abbr[2]+1):
        if n < 10:
            n = f'0{n}'
        borough_labels[borough].append(f'{abbr[1]}{n}')

# First Collection Round

#### All the years for which specific files are available and their corresponding file naming formats

In [57]:
CHP_years = [2018, 2015]  # {year}_CHP_all_data.csv, {year}_Cause_of_premature_death_data.csv
CHS_years = range(2002, 2018)  # chs{year}_public.csv
CD_years = range(1999, 2015)  # cd{year}.csv
CDall_years = range(1999, 2015)  # cdall_{2digityear}.csv
Pov_years = range(2005, 2017)  # NYCgov_Poverty_Measure_Data__{year}_.csv
Child_years = range(2007, 2018)  # AnnualReport{year}.pdf
YRBSS_years = range(2003, 2019, 2) # All archived in file sadc_2017_district.dat

### Youth Risk Behavior Surveilance System Data

In [150]:
# Borough names get cut off
boroughs = {
    'State': 'StatenIsland',
    'Queen': 'Queens',
    'Manha': 'Manhattan',
    'Brook': 'Brooklyn'    
}

for year in YRBSS_years:
    with open('sadc_2017_district.dat', 'r') as source:
        data = ''
        for line in source:
            # It's a national survey, so we extract the NYC-related data
            # Metadata specified in which years NYC participated
            if 'Borough' in line and f' {year} ' in line:
                data += line
    with open('tmp', 'w') as tmp:
        tmp.write(data)
    # File is fixed-width formatted
    df = pd.read_fwf('tmp', header=None)
    # These columns pertain to state, city, and other classifying data I'm not
    # interested in
    df.drop(columns=[0, 1, 2, 4, 5], inplace=True)
    
    # Replacing cut-off borough names mentioned above
    df[3].replace(to_replace=boroughs, inplace=True)
    
    # Nan values are represented as periods...
    df.replace(to_replace='.', value=np.nan, inplace=True)
    df.to_csv(f'data/youthriskbehavior{year}.csv')

"for year in YRBSS_years:\n    with open('sadc_2017_district.dat', 'r') as source:\n        data = ''\n        for line in source:\n            if 'Borough' in line and f' {year} ' in line:\n                data += line\n    with open('tmp', 'w') as tmp:\n        tmp.write(data)\n    df = pd.read_fwf('tmp', header=None)\n    df.drop(columns=[0, 1, 2, 4, 5], inplace=True)\n    df[3].replace(to_replace=boroughs, inplace=True)\n    df.replace(to_replace='.', value=np.nan, inplace=True)\n    df.to_csv(f'data/youthriskbehavior{year}.csv')"

## Find all the PDF files

In [8]:
allfiles = os.listdir('data/NYCDataSources')
pdffiles = [filename for filename in allfiles if filename.endswith('.pdf')]

In [9]:
pdffiles

['infantmortality.pdf',
 'AnnualReport2009.pdf',
 'AnnualReport2008.pdf',
 'AnnualReport2012.pdf',
 'AnnualReport2013.pdf',
 'AnnualReport2007.pdf',
 'AnnualReport2011.pdf',
 'AnnualReport2010.pdf',
 'AnnualReport2014.pdf',
 'AnnualReport2015.pdf',
 'AnnualReport2017.pdf',
 'AnnualReport2016.pdf',
 'AbuseNeglectReport2015To2018.pdf']

### Investigate first the Infant Mortality PDF

In [130]:
file = pdffiles[0]

# Obtained from looking at the pdf
# Tried using multi-indexing for the columns, but tabula
# could not properly parse the headings
columns = ['CommunityDistrict', 'Neighborhoods',
           '2013-2015 InfantMortalityRate',
           '2013-2015 NeonatalMortalityRate',
           '2014-2016 InfantMortalityRate',
           '2014-2016 NeonatalMortalityRate',
           '2015-2017 InfantMortalityRate',
           '2015-2017 NeonatalMortalityRate']

# For this, I manually determined the area and column boundaries
df = read_pdf(file, lattice=True, guess=False, area=[126, 55, 727, 548],
              columns=[111, 247, 297, 348, 399, 449, 499],
              pandas_options={'header': None, 'names': columns})
df.dropna(inplace=True)

df.to_csv(f'data/{file[:-3]}csv', index=False)

### Investigate next the Child Abuse and Neglect Report

In [145]:
file = pdffiles[-1]

# Again tried using multi-indexing for the columns,
# but again resorted to manually specifying column names
columns = ['CommunityDistrict',
           '2015 AbuseNeglectCases',
           '2015 AbuseNeglectChildren',
           '2015 AbuseNeglectIndication',
           '2016 AbuseNeglectCases',
           '2016 AbuseNeglectChildren',
           '2016 AbuseNeglectIndication',
           '2017 AbuseNeglectCases',
           '2017 AbuseNeglectChildren',
           '2017 AbuseNeglectIndication',
           '2018 AbuseNeglectCases',
           '2018 AbuseNeglectChildren',
           '2018 AbuseNeglectIndication']

# And manually determining the area needed
df = read_pdf(file, pages='all', guess=False, area=[111, 18, 459, 765],
              pandas_options={'header': None})
# drop "rank" columns
df.drop(columns=[2,6,10,14], inplace=True)
# tabula put some NaN rows in there just for the fun of it
df.dropna(inplace=True)
df.columns = columns
df.to_csv(f'data/{file[:-3]}csv', index=False)

### Investigate the "AnnualReports" -- Child Welfare and Child Services (Foster Care) Reports

This was an attempt at automating the creation of the column names. Only later did I find out that the vast majority of the boroughs and years were missing many (but not all) of the columns that were straight zeroes down the line...

In [310]:
file = pdffiles[5]

# Each row of headers ia actually read in as a two-row df
df1 = read_pdf(file, pages=19, guess=False, stream=True,
              area=[56, 249, 80, 726],
              columns=[278, 312, 344, 376, 414.8, 446, 483, 517, 552, 586, 622, 657, 695],
              pandas_options={'header': None})
df2 = read_pdf(file, pages=19, guess=False, stream=True,
              area=[104, 249, 126, 731],
              columns=[278, 312, 344, 376, 414.8, 446, 483, 517, 552, 586, 622, 657, 695],
              pandas_options={'header': None})
df3 = read_pdf(file, pages=19, guess=False, stream=True,
              area=[154, 249, 176, 342],
              columns=[279.57, 313],
              pandas_options={'header': None})

df4 = pd.concat([df1, df2, df3], axis=1, ignore_index=True)

# Here is where the top and bottom rows are combined to create a whole df
def combine_labels(col):
    '''
    Takes in the top and bottom row of the df and ensures the new
    column name is of the format 'FosterCareLength({X}yrs-<{Y}yrs)'
    '''
    if col[0][-1] != '-':
        col[0] += '-'
    new_label = f'FosterCareLength({col[0]}{col[1]})'
    return re.sub(r'\s+', '', new_label)

CWlabels = df4.apply(lambda col: combine_labels(col), axis=0).values.tolist()
CWlabels.append('TotalReunifications')
CWlabels.insert(26, 'FosterCareLength(13yrs-<13.5yrs)') # Missing in this particular ds
FClabels = ['TotalPlacements', 'PlacementWithin36mos', '%PlacementsWithin36mos']

# Because of the lack of consistency, I opted to keep the high-frequency (shorter-term)
# labels and just make sure to get a total for all <18yrs
CWlabels_std = CWlabels[:10]

In [556]:
def get_areas(num_words, words, labels):
    '''
    Must be used in conjunction with pdfplumber. pdfplumber's,
    word objects have coordinates, which will be useful with tabula's
    read_pdf function.
    '''
    # Initializations
    start = -1
    areas = []
    
    # Run through each word in succession, checking if it matches 
    # one of the borough's labels
    while start < num_words - 1:
        start += 1
        top = None
        left = None
        bottom = None
        right = None
        df_label = ''
        word = words[start]
        for first in labels:
            # When we find a match, note the top and left coordinates
            # (as the label will be the index)
            if word['text'] == first:
                top = word['top']
                left = word['x0']
                prev_word = words[start-1]
                found = False
                # Run through each borough label in reverse order, to
                # catch the first instance of the highest label for the
                # borough
                for last in reversed(labels):
                    # Start from word following the one we matched previously
                    # to reduce redundancy
                    for end in range(start+1, num_words):
                        word2 = words[end]
                        # When the word is found note the bottom and right
                        # coordinates
                        if word2['text'] == last:
                            bottom = word2['bottom']
                            prev_line = words[end-1]
                            right = prev_line['x1']
                            # Also include some tagging for easier manipulation
                            # when reading and formatting the dataframes
                            if 'Total' in prev_word['text']:
                                df_label = 'total'
                            elif r'%' in prev_line['text']:
                                df_label = 'FC'
                            # So we can break the outside loop
                            found = True
                            # So we don't repeat reading sections
                            start = end
                            break
                    # Breaking the outside loop
                    if found:
                        break
                try:
                    # Give some wiggle room by adding and subtracting from coordinates
                    area = [float(top)-0.5, float(left)-2, float(bottom)+0.5, float(right)+5]
                except:
                    # To help debug
                    print(top, left, bottom, right, word, prev_word, word2, prev_line)
                    raise
                areas.append([[df_label, word['text'], word2['text']], area])
                # Break the final for loop to return to our controlled while loop
                # at the appropriate 'start' value
                break
    return areas

In [557]:
def get_tables(year):
    '''
    Uses get_areas to return the page numbers, labels, and related areas
    for each borough.
    '''
    # Initializations
    borough_dfs = {'Manhattan': {},
                   'Brooklyn': {},
                   'Bronx': {},
                   'Queens': {},
                   'Staten Island': {}}
    file = f'AnnualReport{year}.pdf'
    i = 0
    
    with pdfplumber.open(file) as pdf:
        # Loop through the pages, keep track of page number as it
        # will be used by tabula--which starts numeration at 1
        for page in pdf.pages:
            i += 1
            # Gets all the text from a page as a single string -- easy to check
            # for inclusion of substrings
            text = page.extract_text()
            # Gets all the 'words' as 'word objects' which includes the 
            # text of the word, and the upper, lower, left, and right coordinates
            # for use in get_areas
            words = page.extract_words()
            num_words = len(words)
            
            for borough, labels in borough_labels.items():
                if any(label in text for label in labels):
                    areas = get_areas(num_words, words, labels)
                    borough_dfs[borough][str(i)] = areas
    return borough_dfs

In [558]:
# Initialize dict for misfits (to be able to debug after-the-fact)
strays = {}
# Initialize dict we will actually use to create the larger df
year_dfs = {}
for year in range(2007, 2018):
    borough_dfs = get_tables(year)
    year_dfs[str(year)] = borough_dfs

In [559]:
# Take a peek
year_dfs

{'2007': {'Manhattan': {'22': [[['', 'MN01', 'MN12'],
     [381.09, 72.16, 508.304, 720.511]]],
   '23': [[['', 'MN01', 'MN12'], [96.45, 72.16, 222.943, 720.509]],
    [['total', 'MN01', 'MN12'], [276.93, 72.16, 404.143, 373.398]]],
   '28': [[['FC', 'MN01', 'MN12'], [136.53, 141.04, 275.023, 579.957]]]},
  'Brooklyn': {'21': [[['', 'BK01', 'BK18'], [91.89, 73.12, 298.785, 720.509]],
    [['', 'BK01', 'BK18'], [368.13, 73.12, 574.305, 720.509]]],
   '22': [[['total', 'BK01', 'BK18'], [85.17, 73.12, 291.345, 370.437]]],
   '27': [[['FC', 'BK01', 'BK18'], [330.93, 141.04, 537.105, 579.961]]]},
  'Bronx': {'19': [[['', 'BX01', 'BX12'], [257.49, 71.92, 395.987, 720.5]]],
   '20': [[['', 'BX01', 'BX12'], [85.17, 71.92, 223.666, 720.503]],
    [['total', 'BX01', 'BX12'], [284.37, 71.92, 422.866, 370.444]]],
   '27': [[['FC', 'BX01', 'BX12'], [150.45, 141.04, 288.946, 579.953]]]},
  'Queens': {'24': [[['', 'QN01', 'QN14'], [96.45, 72.4, 257.505, 720.509]],
    [['', 'QN01', 'QN14'], [310.77, 

##### The 2017 right-bounding coordinates for pages 17 and 18 can't be correct. Let's change them to 721.

In [574]:
latest = year_dfs['2017']
for borough, dfs in latest.items():
    try:
        info = dfs['18'][0]
        page = 18
    except:
        info = dfs['17'][0]
        page = 17
    area = info[1]
    area[3] = 721

In [560]:
def get_borough_df(file, borough, df_pages, year, strays):
    '''
    Function to get a unified borough-wide df from each year.
    Adds to strays if necessary.
    '''
    # Initialization
    to_merge = []
    first = True
    k = 0
    # dfs is a dictionary with keys that are the page numbers
    # and values that are the objects returned by get_areas
    for i, info in df_pages.items():
        page = int(i)
        # Just to keep track of same borough appearing twice on one page
        j = 0
        for each in info:
            j += 1
            k += 1
            df_label = each[0][0]
            begin = each[0][1]
            end = each[0][2]
            area = each[1]
            # tabula doing its work
            df = read_pdf(file, pages=page, guess=False, area=area,
                          pandas_options={'header': None, 'index_col': 0})
            try:
                n_cols = df.shape[1] - 1
            except:
                print(f'could not READ {borough}{year}_{i}{j}')
            # For the dfs labeled 'total', I am only interested in the last column,
            # the rest are sparse. If parents/family are unable to regain custody
            # of the child after a few years, it is unlikely they will ever do so.
            if df_label == 'total':
                try:
                    df = df.iloc[:, -1]
                    srs = df.rename(f'{year}TotalReunifications')
                except:
                    print(f'could not RENAME TotalReunifications {borough}{year}_{i}{j}')
                    strays[borough+str(year)+'_'+i+str(j)] = df
                    continue
                else:
                    to_merge.append(srs)
                    continue
            # For the label 'FC', these are children returning to foster care--with
            # a focus on those returning within the last 3 years
            elif df_label == 'FC' or (year != '2007' and k == 1):
                try:
                    df.drop(columns=[1], inplace=True)
                    df.columns = [f'{year}{FC}' for FC in FClabels]
                except:
                    print(f'could not RENAME FC {borough}{year}_{i}{j}')
                    strays[borough+str(year)+'_'+i+str(j)] = df
                    continue
                else:
                    to_merge.append(df)
                    continue
            elif not first:
                print(f'DROPPED {borough}{year}_{i}{j}')
                strays[borough+str(year)+'_'+i+str(j)] = df
                continue
            elif df[2].isnull().sum() > 1:
                try:
                    df[2] = df.apply(lambda row: [
                        n for n in row[1].split(' ') if n.isnumeric()], axis=1)
                except:
                    print(f'could not COPY infant data {borough}{year}_{i}{j}')
                    strays[borough+str(year)+'_'+i+str(j)] = df
                    continue
            if end == borough_labels[borough][-1]:
                first = False
            df.drop(columns=[1], inplace=True)
            try:
                df = df.iloc[:, :10]
                df.columns = [f'{year}{CW}' for CW in CWlabels_std]
            except:
                print(f'could not RENAME {borough}{year}_{i}{j}')
                strays[borough+str(year)+'_'+i+str(j)] = df
                continue
            else:
                to_merge.append(df)
    try:
        df = pd.concat(to_merge, axis=1, join='outer', sort=True)
    except:
        print(f'unable to MERGE {year} {borough}')
        strays[borough+str(year)] = to_merge
        return None, strays
    return df, strays

In [596]:
# Initialization
y_to_merge = []

# year_dfs is a dict with years as keys and objects returned by get_tables
# as values
for year, borough_dfs in year_dfs.items():
    file = f'AnnualReport{year}.pdf'
    b_to_merge = []

    # Those objects are also a dict with keys as borough names and values
    # as dicts with keys of page numbers and values of df info as returned
    # by get_areas
    for borough, df_pages in borough_dfs.items():
        b_df, strays = get_borough_df(file, borough, df_pages, year, strays)
        b_to_merge.append(b_df)
    try:
        df = pd.concat(b_to_merge, axis=0, join='outer', sort=True)
    except:
        print(f'unable to merge ALL of {year}')
        strays[str(year)] = b_to_merge
    else:
        y_to_merge.append(df)

df = pd.concat(y_to_merge, axis=1, join='outer', sort=True)
df.to_csv(f'data/ChildWelfare.csv', index=True)

DROPPED Manhattan Manhattan2007_231
DROPPED Brooklyn Brooklyn2007_212
DROPPED Bronx Bronx2007_201
DROPPED Queens Queens2007_242
DROPPED Staten Island Staten Island2007_252
DROPPED Manhattan Manhattan2008_241
DROPPED Brooklyn Brooklyn2008_222
DROPPED Bronx Bronx2008_211
DROPPED Queens Queens2008_252
DROPPED Staten Island Staten Island2008_262
DROPPED Manhattan Manhattan2009_241
DROPPED Brooklyn Brooklyn2009_222
DROPPED Bronx Bronx2009_211
DROPPED Queens Queens2009_252
DROPPED Staten Island Staten Island2009_262
DROPPED Manhattan Manhattan2010_241
DROPPED Brooklyn Brooklyn2010_222
DROPPED Bronx Bronx2010_211
DROPPED Queens Queens2010_252
DROPPED Staten Island Staten Island2010_262
unable to merge ALL of 2012


#### Everything that got dropped was supposed to.

#### First I'll look into why the 2012 merge failed

In [597]:
bad_year = strays['2012']
for df in bad_year:
    print(df.index, df.columns)
    print(len(df.columns))

Index(['MN01', 'MN02', 'MN03', 'MN04', 'MN05', 'MN06', 'MN07', 'MN08', 'MN09',
       'MN10', 'MN11', 'MN12'],
      dtype='object') Index(['2012TotalPlacements', '2012PlacementWithin36mos',
       '2012%PlacementsWithin36mos', '2012FosterCareLength(losgr-<6mo.)',
       '2012FosterCareLength(6mo-<1yr)', '2012FosterCareLength(1yr-<1.5yrs)',
       '2012FosterCareLength(1.5yrs-<2yrs)',
       '2012FosterCareLength(2yrs-<2.5yrs)',
       '2012FosterCareLength(2.5yrs-<3yrs)',
       '2012FosterCareLength(3yrs-<3.5yrs)',
       '2012FosterCareLength(3.5yrs-<4yrs)',
       '2012FosterCareLength(4yrs-<4.5yrs)',
       '2012FosterCareLength(4.5yrs-<5yrs)',
       '2012FosterCareLength(losgr-<6mo.)', '2012FosterCareLength(6mo-<1yr)',
       '2012FosterCareLength(1yr-<1.5yrs)',
       '2012FosterCareLength(1.5yrs-<2yrs)',
       '2012FosterCareLength(2yrs-<2.5yrs)',
       '2012FosterCareLength(2.5yrs-<3yrs)',
       '2012FosterCareLength(3yrs-<3.5yrs)',
       '2012FosterCareLength(3.5yrs-<4yr

Seems we have repeats in Manhattan

In [598]:
badMN = deepcopy(bad_year[0])
badMN

,2012TotalPlacements,2012PlacementWithin36mos,2012%PlacementsWithin36mos,2012FosterCareLength(losgr-<6mo.),2012FosterCareLength(6mo-<1yr),2012FosterCareLength(1yr-<1.5yrs),2012FosterCareLength(1.5yrs-<2yrs),2012FosterCareLength(2yrs-<2.5yrs),2012FosterCareLength(2.5yrs-<3yrs),2012FosterCareLength(3yrs-<3.5yrs),...,2012FosterCareLength(6mo-<1yr),2012FosterCareLength(1yr-<1.5yrs),2012FosterCareLength(1.5yrs-<2yrs),2012FosterCareLength(2yrs-<2.5yrs),2012FosterCareLength(2.5yrs-<3yrs),2012FosterCareLength(3yrs-<3.5yrs),2012FosterCareLength(3.5yrs-<4yrs),2012FosterCareLength(4yrs-<4.5yrs),2012FosterCareLength(4.5yrs-<5yrs),2012TotalReunifications
MN01,5,0,0.0%,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3
MN02,2,0,0.0%,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
MN03,123,23,18.7%,44.0,15.0,10.0,10.0,2.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,88
MN04,36,1,2.8%,9.0,3.0,1.0,0.0,1.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15
MN05,10,1,10.0%,3.0,3.0,1.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8
MN06,11,2,18.2%,1.0,1.0,2.0,1.0,2.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9
MN07,48,6,12.5%,6.0,3.0,2.0,3.0,0.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19
MN08,23,5,21.7%,7.0,0.0,2.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9
MN09,93,12,12.9%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,19.0,4.0,2.0,2.0,3.0,0.0,1.0,0.0,3.0,58
MN10,188,23,6.4%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,29.0,6.0,5.0,4.0,6.0,3.0,1.0,0.0,5.0,119


Ah, must have been a ds split across two pages

In [599]:
FC = badMN.columns[:3]
CWtot = badMN.columns[-1]

In [600]:
fc_df = badMN[FC]
cwtot_df = badMN[CWtot]
badMN.drop(columns=FC, inplace=True)
badMN.drop(columns=CWtot, inplace=True)
badMN

,2012FosterCareLength(losgr-<6mo.),2012FosterCareLength(6mo-<1yr),2012FosterCareLength(1yr-<1.5yrs),2012FosterCareLength(1.5yrs-<2yrs),2012FosterCareLength(2yrs-<2.5yrs),2012FosterCareLength(2.5yrs-<3yrs),2012FosterCareLength(3yrs-<3.5yrs),2012FosterCareLength(3.5yrs-<4yrs),2012FosterCareLength(4yrs-<4.5yrs),2012FosterCareLength(4.5yrs-<5yrs),2012FosterCareLength(losgr-<6mo.),2012FosterCareLength(6mo-<1yr),2012FosterCareLength(1yr-<1.5yrs),2012FosterCareLength(1.5yrs-<2yrs),2012FosterCareLength(2yrs-<2.5yrs),2012FosterCareLength(2.5yrs-<3yrs),2012FosterCareLength(3yrs-<3.5yrs),2012FosterCareLength(3.5yrs-<4yrs),2012FosterCareLength(4yrs-<4.5yrs),2012FosterCareLength(4.5yrs-<5yrs)
MN01,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
MN02,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
MN03,44.0,15.0,10.0,10.0,2.0,1.0,1.0,2.0,1.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
MN04,9.0,3.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
MN05,3.0,3.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
MN06,1.0,1.0,2.0,1.0,2.0,0.0,0.0,0.0,0.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
MN07,6.0,3.0,2.0,3.0,0.0,1.0,1.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
MN08,7.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
MN09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23.0,19.0,4.0,2.0,2.0,3.0,0.0,1.0,0.0,3.0
MN10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,59.0,29.0,6.0,5.0,4.0,6.0,3.0,1.0,0.0,5.0


In [601]:
badMN_bot = badMN.loc[badMN.index[-4:]].dropna(axis=1)
badMN_top = badMN.loc[badMN.index[:-4]].dropna(axis=1)
goodMN = pd.concat([badMN_top, badMN_bot], join='outer', sort=True)
goodMN

,2012FosterCareLength(1.5yrs-<2yrs),2012FosterCareLength(1yr-<1.5yrs),2012FosterCareLength(2.5yrs-<3yrs),2012FosterCareLength(2yrs-<2.5yrs),2012FosterCareLength(3.5yrs-<4yrs),2012FosterCareLength(3yrs-<3.5yrs),2012FosterCareLength(4.5yrs-<5yrs),2012FosterCareLength(4yrs-<4.5yrs),2012FosterCareLength(6mo-<1yr),2012FosterCareLength(losgr-<6mo.)
MN01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0
MN02,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
MN03,10.0,10.0,1.0,2.0,2.0,1.0,2.0,1.0,15.0,44.0
MN04,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,3.0,9.0
MN05,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,3.0,3.0
MN06,1.0,2.0,0.0,2.0,0.0,0.0,2.0,0.0,1.0,1.0
MN07,3.0,2.0,1.0,0.0,0.0,1.0,0.0,0.0,3.0,6.0
MN08,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0
MN09,2.0,4.0,3.0,2.0,1.0,0.0,3.0,0.0,19.0,23.0
MN10,5.0,6.0,6.0,4.0,1.0,3.0,5.0,0.0,29.0,59.0


In [605]:
goodMN_full = pd.concat([fc_df, goodMN, cwtot_df], axis=1, join='outer', sort=True)
bad_year[0] = goodMN_full
for df in bad_year:
    print(df.index, df.columns)
    print(len(df.columns))

Index(['MN01', 'MN02', 'MN03', 'MN04', 'MN05', 'MN06', 'MN07', 'MN08', 'MN09',
       'MN10', 'MN11', 'MN12'],
      dtype='object') Index(['2012TotalPlacements', '2012PlacementWithin36mos',
       '2012%PlacementsWithin36mos', '2012FosterCareLength(1.5yrs-<2yrs)',
       '2012FosterCareLength(1yr-<1.5yrs)',
       '2012FosterCareLength(2.5yrs-<3yrs)',
       '2012FosterCareLength(2yrs-<2.5yrs)',
       '2012FosterCareLength(3.5yrs-<4yrs)',
       '2012FosterCareLength(3yrs-<3.5yrs)',
       '2012FosterCareLength(4.5yrs-<5yrs)',
       '2012FosterCareLength(4yrs-<4.5yrs)', '2012FosterCareLength(6mo-<1yr)',
       '2012FosterCareLength(losgr-<6mo.)', '2012TotalReunifications'],
      dtype='object')
14
Index(['BK01', 'BK02', 'BK03', 'BK04', 'BK05', 'BK06', 'BK07', 'BK08', 'BK09',
       'BK10', 'BK11', 'BK12', 'BK13', 'BK14', 'BK15', 'BK16', 'BK17', 'BK18'],
      dtype='object', name=0) Index(['2012TotalPlacements', '2012PlacementWithin36mos',
       '2012%PlacementsWithin36mos', '2012

In [612]:
ChildWelfare2012 = pd.concat(bad_year, join='outer', sort=True)
CWminus2012 = pd.read_csv('data/ChildWelfare.csv', index_col=0)
full_df = pd.concat([CWminus2012, ChildWelfare2012],
                    axis=1, join='outer', sort=True)
full_df.drop(index=['BK 11'], inplace=True)
full_df.to_csv('data/ChildWelfare.csv', index=True)
full_df

,2007%PlacementsWithin36mos,2007FosterCareLength(1.5yrs-<2yrs),2007FosterCareLength(1yr-<1.5yrs),2007FosterCareLength(2.5yrs-<3yrs),2007FosterCareLength(2yrs-<2.5yrs),2007FosterCareLength(3.5yrs-<4yrs),2007FosterCareLength(3yrs-<3.5yrs),2007FosterCareLength(4.5yrs-<5yrs),2007FosterCareLength(4yrs-<4.5yrs),2007FosterCareLength(6mo-<1yr),...,2012FosterCareLength(2yrs-<2.5yrs),2012FosterCareLength(3.5yrs-<4yrs),2012FosterCareLength(3yrs-<3.5yrs),2012FosterCareLength(4.5yrs-<5yrs),2012FosterCareLength(4yrs-<4.5yrs),2012FosterCareLength(6mo-<1yr),2012FosterCareLength(losgr-<6mo.),2012PlacementWithin36mos,2012TotalPlacements,2012TotalReunifications
BK01,5.0%,5.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,3.0,9.0,17,8.0,43.0,41.0
BK02,0.0%,0.0,2.0,0.0,2.0,1.0,0.0,3.0,0.0,10.0,...,1.0,0.0,0.0,0.0,0.0,5.0,15,4.0,48.0,29.0
BK03,4.3%,2.0,8.0,0.0,3.0,0.0,5.0,0.0,2.0,24.0,...,11.0,2.0,4.0,1.0,2.0,31.0,59,46.0,193.0,150.0
BK04,1.2%,3.0,11.0,0.0,1.0,0.0,2.0,0.0,0.0,23.0,...,3.0,3.0,6.0,0.0,0.0,16.0,32,17.0,99.0,75.0
BK05,0.8%,5.0,14.0,4.0,2.0,2.0,6.0,5.0,4.0,18.0,...,14.0,4.0,1.0,3.0,3.0,31.0,53,41.0,235.0,157.0
BK06,2.6%,1.0,3.0,1.0,0.0,1.0,0.0,0.0,0.0,7.0,...,1.0,4.0,1.0,0.0,0.0,6.0,11,11.0,35.0,29.0
BK07,0.0%,0.0,5.0,0.0,1.0,3.0,3.0,0.0,0.0,3.0,...,0.0,1.0,0.0,2.0,0.0,3.0,15,2.0,49.0,26.0
BK08,2.5%,1.0,7.0,2.0,1.0,2.0,0.0,1.0,3.0,12.0,...,4.0,3.0,5.0,0.0,0.0,13.0,6,8.0,71.0,43.0
BK09,5.0%,1.0,5.0,4.0,0.0,0.0,1.0,0.0,0.0,3.0,...,1.0,0.0,0.0,0.0,0.0,6.0,11,6.0,37.0,22.0
BK10,0.0%,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,5,6.0,20.0,8.0


# Second Collection Round

In [4]:
directory = 'data/NYCDataSourcesRound2'
savedir = 'data/raw_csvs/'
allfiles_2 = os.listdir(directory)
# pdffiles = [filename for filename in allfiles if filename.endswith('.pdf')]
csvfiles = [filename for filename in allfiles_2 if filename.endswith('.csv')]
excelfiles = [filename for filename in allfiles_2 if filename.endswith('.xls') or filename.endswith('.xlsx')]

print('CSV Files:')
print(csvfiles)
print('\nExcel Files:')
print(excelfiles)

CSV Files:
['communitydistrict-studentsperformingatgradelevelinmath4thgrade.csv', 'communitydistrict-studentsperformingatgradelevelinenglishlanguagearts4thgrade.csv', 'nysd.csv', 'ALL_VARIABLES_D2G1.csv', '2005_-_2011_Graduation_Outcomes_-_School_Level_-_SWD.csv', '2005_-_2011_Graduation_Outcomes_-_School_Level_-_-_Gender.csv', '2005_-_2011_Graduation_Outcomes_-_School_Level_-_-_Ethnicity.csv', 'sub-borougharea-disconnectedyouth.csv', '2001-2013_Graduation_Outcome_School_Level_-_ALL_ELL_SWD_ETHNICITY_GENDER_EVER_ELL_TRANSFER_SCHOOLS.csv', '2017_-_2018_Graduation_Outcomes_School.csv', '2005-2015_Graduation_Rates_Public_School_-_APM.csv', '2005-2010_Graduation_Outcomes_-_School_Level.csv', '2009_-_2010_Graduation_Outcomes_-_Regents-based_Math-_ELA_APM_-_School_Level.csv', '2005_-_2011_Graduation_Outcomes_-_School_Level_-_Classes_of_-_Total_Cohort.csv', 'METADATA_DATA2GO1.csv', '2005_-_2011_Graduation_Outcomes_-_School_Level-_ELL.csv']

Excel Files:
['Data2go.nyc.2nd.Edition.All.Variables

In [6]:
fourthgrade_stats = csvfiles[0:2]
dfs = []

for file in fourthgrade_stats:
    path = f'{directory}/{file}'
    df = pd.read_csv(os.path.join(directory, file), index_col=2)
    pre = df.iloc[0,0]
    df.drop(columns=df.columns[:2], inplace=True)
    df.columns = [pre+col for col in df.columns]
    dfs.append(df)

df = pd.concat(dfs, axis=1, sort=True)
df.to_csv(savedir+'proficiency_4thgrade')

In [146]:
excelfiles

['Data2go.nyc.2nd.Edition.All.Variables.xlsx',
 'On-Time_HS_Grad.xlsx',
 'FC_subsidized_housing_database_2018-06-27.xlsx',
 'Data2Go.Full.Dataset 2018.xlsx',
 '~$Data2go.nyc.2nd.Edition.All.Variables.xlsx',
 'elsec17t.xls',
 'MappingAmerica_Demographics.xlsx',
 'nta-metadata (1).xlsx',
 'DATA2GOHEALTH.NYC 1st edition.xlsx',
 'variable-list-labs.xls',
 'elsec17_sumtables.xls']

In [86]:
df = pd.read_excel(os.path.join(directory, excelfiles[3]),
                   header=None, skiprows=4, nrows=1, squeeze=True)
types = df.values[0]
print(set(types))

include = []
for count in range(len(types)):
    if types[count] == 'Community District' or types[count] == 'PUMA':
        include.append(count)

include.insert(0,0)

{nan, 'Borough', 'NYC', 'PUMA', 'Community District', 'Tract', 'UNIT OF GEOGRAPHY'}


[212,
 213,
 214,
 215,
 216,
 217,
 218,
 219,
 220,
 221,
 222,
 223,
 224,
 225,
 226,
 227,
 228,
 229,
 230,
 231,
 232,
 233,
 234,
 235,
 236,
 237,
 238,
 239,
 240,
 241,
 242,
 243,
 244,
 245,
 246,
 247,
 248,
 249,
 250,
 251,
 252,
 253,
 254,
 255,
 256,
 257,
 258,
 259,
 260,
 261,
 262,
 263,
 264,
 265,
 266,
 267,
 268,
 269,
 270,
 271,
 272,
 273,
 274,
 275,
 276,
 277,
 278,
 279,
 280,
 281,
 282,
 283,
 284,
 285,
 286,
 287,
 288,
 289,
 290,
 291,
 292,
 293,
 294,
 295,
 296,
 297,
 298,
 299,
 300,
 301,
 302,
 303,
 304,
 305,
 306,
 307,
 308,
 309,
 310,
 311,
 312,
 313,
 314,
 315,
 316,
 317,
 318,
 319,
 320,
 321,
 322,
 323,
 324,
 325,
 326,
 327,
 328,
 329,
 330,
 428,
 448,
 449,
 450,
 451,
 452,
 453,
 454,
 455,
 456,
 499,
 500,
 501,
 502,
 503,
 504,
 505,
 506,
 507,
 508,
 509,
 510,
 511,
 512,
 513,
 514,
 515,
 516,
 517,
 518,
 519,
 555,
 556,
 557,
 558,
 559,
 560,
 561,
 562,
 563,
 564,
 565,
 572,
 573,
 574,
 692,
 693,
 694

In [93]:
df = pd.read_excel(os.path.join(directory, excelfiles[3]), usecols=include, header=None,
                   skiprows=[0,1,2,3,4,5,7,8,9,11,12,13,14,15,16,17,18,19], nrows=116)

In [98]:
def get_col_names_year(df, col_names=['GEO LABEL']):
    for col in range(1, len(df.columns)):
        name = ' '.join([df.iloc[0, col], str(df.iloc[1, col])])
        col_names.append(name)
    
    df.columns = col_names
    df.drop(index=[0,1], inplace=True)


In [95]:
get_col_names_year(df)

comm_dist = [('Community District' in label) for label in df['GEO LABEL']]
CD = [('CD' in label) for label in df['GEO LABEL']]

df1 = df[comm_dist]
df2 = df[CD]

dfs = [df1, df2]

boroughs_['Richmond'] = boroughs_['Staten Island']
boroughs_['Staten'] = boroughs_['Staten Island']

def get_CD_nums(label, boroughs=boroughs_):
    parts = label.split()
    borough_code = boroughs[parts[0]][2]
    nums = [int(part) for part in parts if part.isnumeric()]
    if len(nums) == 1:
        new_label = str(nums[0] + borough_code)
    else:
        new_label = ' '.join([str(num + borough_code) for num in nums])
        
    return new_label

def set_df_index(df):
    df.dropna(axis=1, how='all', inplace=True)
    df['GEO LABEL'] = df['GEO LABEL'].apply(get_CD_nums)
    df['Community District'] = df['GEO LABEL'].values
    df.set_index('GEO LABEL', inplace=True)

for df in dfs:
    set_df_index(df)

df = pd.concat(dfs, axis=1, join='outer', sort=True)

df.to_csv(savedir+'d2g1.csv')

In [155]:
def dict_or_int(value, initialized=False):
    if not initialized:
        if isinstance(value, int):
            output = list(range(value))
        else:
            output = {}
            for sheet, num in value.items():
                output[sheet] = list(range(num))
        return output
    else:
        if isinstance(value, int):
            if isinstance(initialized, list):
                initialized.remove(value)
            else:
                for sheet in initialized.keys():
                    initialized[sheet].remove(value)
        else:
            if isinstance(initialized, list):
                output = {}
                for sheet, num in value.items():
                    initial = deepcopy(initialized)
                    initial.remove(num)
                    output[sheet] = initial
                return output
            else:
                for sheet in initialized.keys():
                    initialized[sheet].remove(value[sheet])
        return initialized

def read_d2g_new(filepath, name_row, year_row, startrow, geolabel_col, startcol):
    skiprows = dict_or_int(startrow)
    skiprows = dict_or_int(name_row, initialized=skiprows)
    skiprows = dict_or_int(year_row, initialized=skiprows)
    
    dropcols = dict_or_int(startcol)
    dropcols = dict_or_int(geolabel_col, initialized=dropcols)

    dfs = {}
    
    try:
        for sheet, rows in skiprows.items():
            df = pd.read_excel(filepath, sheet_name=sheet, header=None, skiprows=rows)
            dfs[sheet] = df
    except:
        for sheet in ['CD', 'Puma']:
            df = pd.read_excel(filepath, sheet_name=sheet, header=None, skiprows=skiprows)
            dfs[sheet] = df

    try:
        for sheet, cols in dropcols.items():
            dfs[sheet].drop(columns=cols, inplace=True)
    except:
        for df in dfs.values():
            df.drop(columns=dropcols, inplace=True)
    
    for df in dfs.values():
        get_col_names_year(df, col_names=['GEO LABEL'])
        set_df_index(df)
    
    df = pd.concat(dfs.values(), axis=1, join='outer', sort=True)
    return df

In [152]:
file = os.path.join(directory, excelfiles[0])

df = read_d2g_new(file, 8, 13, 17, 1, {'CD': 6, 'Puma':5})

df.to_csv(savedir +'d2g2.csv')

In [156]:
file = os.path.join(directory, excelfiles[8])

df = read_d2g_new(file, {'CD': 6, 'Puma': 7}, {'CD': 10, 'Puma': 11},
                  {'CD': 14, 'Puma':15}, 2, 7)

df.to_csv(savedir + 'd2ghealth.csv')

In [6]:
csvfiles

['communitydistrict-studentsperformingatgradelevelinmath4thgrade.csv',
 'communitydistrict-studentsperformingatgradelevelinenglishlanguagearts4thgrade.csv',
 'nysd.csv',
 'ALL_VARIABLES_D2G1.csv',
 '2005_-_2011_Graduation_Outcomes_-_School_Level_-_SWD.csv',
 '2005_-_2011_Graduation_Outcomes_-_School_Level_-_-_Gender.csv',
 '2005_-_2011_Graduation_Outcomes_-_School_Level_-_-_Ethnicity.csv',
 'sub-borougharea-disconnectedyouth.csv',
 '2001-2013_Graduation_Outcome_School_Level_-_ALL_ELL_SWD_ETHNICITY_GENDER_EVER_ELL_TRANSFER_SCHOOLS.csv',
 '2017_-_2018_Graduation_Outcomes_School.csv',
 '2005-2015_Graduation_Rates_Public_School_-_APM.csv',
 '2005-2010_Graduation_Outcomes_-_School_Level.csv',
 '2009_-_2010_Graduation_Outcomes_-_Regents-based_Math-_ELA_APM_-_School_Level.csv',
 '2005_-_2011_Graduation_Outcomes_-_School_Level_-_Classes_of_-_Total_Cohort.csv',
 'METADATA_DATA2GO1.csv',
 '2005_-_2011_Graduation_Outcomes_-_School_Level-_ELL.csv']

In [8]:
df = pd.read_csv(os.path.join(directory, csvfiles[10]))
df

,DBN,School,Cohort Year,Cohort Category,Dem,# APM Cohort,# number of graduates,# achieving APM,% of cohort,% of graduates
0,01M056,CORLEARS SCHOOL,2008,4 Year August,NaN,1,s,s,s,s
1,01M292,HENRY STREET SCHOOL FOR INTERNATIONAL STUDIES,2006,4 Year August,NaN,78,44,7,9.0%,15.9%
2,01M292,HENRY STREET SCHOOL FOR INTERNATIONAL STUDIES,2007,4 Year August,NaN,77,45,5,6.5%,11.1%
3,01M292,HENRY STREET SCHOOL FOR INTERNATIONAL STUDIES,2008,4 Year August,NaN,70,36,3,4.3%,8.3%
4,01M292,HENRY STREET SCHOOL FOR INTERNATIONAL STUDIES,2009,4 Year August,NaN,85,49,1,1.2%,2.0%
5,01M292,HENRY STREET SCHOOL FOR INTERNATIONAL STUDIES,2010,4 Year August,NaN,61,26,2,3.3%,7.7%
6,01M292,HENRY STREET SCHOOL FOR INTERNATIONAL STUDIES,2011,4 Year August,NaN,73,46,2,2.7%,4.3%
7,01M448,UNIVERSITY NEIGHBORHOOD HIGH SCHOOL,2006,4 Year August,NaN,124,60,17,13.7%,28.3%
8,01M448,UNIVERSITY NEIGHBORHOOD HIGH SCHOOL,2007,4 Year August,NaN,150,106,19,12.7%,17.9%
9,01M448,UNIVERSITY NEIGHBORHOOD HIGH SCHOOL,2008,4 Year August,NaN,120,90,18,15.0%,20.0%
